In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
import re
from collections import defaultdict
from rapidfuzz import process, fuzz


In [3]:
# Load the dataset
data_path = 'GraduateEmploymentSurvey.csv'
df = pd.read_csv(data_path)

# Display the first few rows
print("Dataset Head:")
print(df.head())

# Check dataset info and summary statistics
print("\nDataset Info:")
print(df.info())
print("\nSummary Statistics:")
print(df.describe())



Dataset Head:
   year                        university  \
0  2013  Nanyang Technological University   
1  2013  Nanyang Technological University   
2  2013  Nanyang Technological University   
3  2013  Nanyang Technological University   
4  2013  Nanyang Technological University   

                                          school  \
0  College of Business (Nanyang Business School)   
1  College of Business (Nanyang Business School)   
2  College of Business (Nanyang Business School)   
3  College of Business (Nanyang Business School)   
4                         College of Engineering   

                                        degree employment_rate_overall  \
0                     Accountancy and Business                    97.4   
1  Accountancy (3-yr direct Honours Programme)                    97.1   
2     Business (3-yr direct Honours Programme)                    90.9   
3                       Business and Computing                    87.5   
4                        Aerospa

In [4]:
print("Columns:", df.columns.tolist())


Columns: ['year', 'university', 'school', 'degree', 'employment_rate_overall', 'employment_rate_ft_perm', 'basic_monthly_mean', 'basic_monthly_median', 'gross_monthly_mean', 'gross_monthly_median', 'gross_mthly_25_percentile', 'gross_mthly_75_percentile']


## SMU change 4 years to 4 year programme
## NTU change and remove double degree

In [5]:
mask_smu = (df["university"] == "Singapore Management University") & (
    df["school"].str.contains("Accountancy|Business", case=False)
)

# Vectorized updates for SMU:
mask_acc_laude = (
    mask_smu
    & df["degree"].str.contains("Accountancy", case=False)
    & df["degree"].str.contains("Laude", case=False)
)
df.loc[mask_acc_laude, "degree"] = "Accountancy Cum Laude and above"

mask_bus_laude = (
    mask_smu
    & df["degree"].str.contains("Business", case=False)
    & df["degree"].str.contains("Laude", case=False)
)
df.loc[mask_bus_laude, "degree"] = "Business Management Cum Laude and above"

mask_acc = (
    mask_smu
    & df["degree"].str.contains("Accountancy", case=False)
    & ~df["degree"].str.contains("Laude", case=False)
)
df.loc[mask_acc, "degree"] = "Accountancy"

mask_bus = (
    mask_smu
    & df["degree"].str.contains("Business", case=False)
    & ~df["degree"].str.contains("Laude", case=False)
)
df.loc[mask_bus, "degree"] = "Business Management"

# ----- NTU: Remove double degree rows and rename degrees -----
# mask_double = (
#     (df['university'] == 'Nanyang Technological University') &
#     (df['school'] == 'College of Business (Nanyang Business School)') &
#     (df['degree'].str.contains('and', case=False))
# )
# df = df[~mask_double]
mask_double = (
    (df["university"] == "Nanyang Technological University") &
    (df["degree"].str.contains("and", case=False, na=False))
)

# 2) Show how many rows match this condition
num_double_degrees = mask_double.sum()
print("Number of NTU double-degree rows:", num_double_degrees)

# 3) Display those rows before updating
double_degree_rows = df.loc[mask_double, ["university", "school", "degree"]]
print("\nNTU double-degree rows (before update):")
print(double_degree_rows)

# 4) Truncate the degree to only the part before "and"
df.loc[mask_double, "degree"] = (
    df.loc[mask_double, "degree"]
    .str.lower()
    .str.split("and", n=1)  # split on the first occurrence of "and"
    .str[0]
    .str.strip()
    .str.title()
)

mask_acc_ntu = (
    (df["university"] == "Nanyang Technological University")
    & (df["school"] == "College of Business (Nanyang Business School)")
    & (df["degree"].str.contains("accountancy", case=False))
)
df.loc[mask_acc_ntu, "degree"] = "Bachelor of Accountancy (Hons)"

mask_bus_ntu = (
    (df["university"] == "Nanyang Technological University")
    & (df["school"] == "College of Business (Nanyang Business School)")
    & (df["degree"].str.contains("business", case=False))
)
df.loc[mask_bus_ntu, "degree"] = "Bachelor of Business (Hons)"

df["university"] = df["university"].replace(
    [
        "Nanyang Technological University",
        "National University of Singapore",
        "Singapore Management University",
        "Singapore Institute of Technology",
        "Singapore University of Technology and Design",
        "Singapore University of Social Sciences",
    ],
    ["NTU", "NUS", "SMU", "SIT", "SUTD", "SUSS"],
)

# Check to ensure the result
df["university"].unique()

# ----- Save the updated DataFrame to CSV and Excel -----
df.to_csv("standardized_degrees.csv", index=False)

# Orignally the data has na we change this to np.nan so it properly reflects that it is not a number
df = df.replace("na", np.nan)

# Display the unique updated degree names
print("Updated degrees:")
print(df["degree"].unique())

Number of NTU double-degree rows: 121

NTU double-degree rows (before update):
                            university  \
0     Nanyang Technological University   
3     Nanyang Technological University   
6     Nanyang Technological University   
10    Nanyang Technological University   
12    Nanyang Technological University   
...                                ...   
1193  Nanyang Technological University   
1194  Nanyang Technological University   
1196  Nanyang Technological University   
1197  Nanyang Technological University   
1200  Nanyang Technological University   

                                             school  \
0     College of Business (Nanyang Business School)   
3     College of Business (Nanyang Business School)   
6                            College of Engineering   
10                           College of Engineering   
12                           College of Engineering   
...                                             ...   
1193                           

In [6]:
ntu_degrees = df[(df['university'] == 'NTU') & 
                 (df['school'] == 'College of Business (Nanyang Business School)')]['degree'].unique()
print("NTU Degrees:", ntu_degrees)

NTU Degrees: ['Bachelor of Accountancy (Hons)' 'Bachelor of Business (Hons)']


In [8]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report


# --- Step 2: Clean and normalize the degree text ---
def clean_degree_for_model(text):
    if pd.isnull(text):
        return ""
    text = text.lower().strip()
    # Remove unwanted symbols like *, #, ^
    text = re.sub(r'[\*\#\^]+', '', text)
    # Collapse multiple spaces into one
    text = re.sub(r'\s+', ' ', text)
    # Remove honours markers, e.g., "(hons)" or "honours"
    text = re.sub(r'\(hons\)', '', text)
    text = re.sub(r'\shonours', '', text)
    # Remove programme details (anything in parentheses mentioning 'programme')
    text = re.sub(r'\(.*programme\)', '', text)
    return text.strip()

df['degree_cleaned'] = df['degree'].apply(clean_degree_for_model)

# --- Step 3: Create a binary honours flag ---
df['is_honours'] = df['degree'].str.contains('hons|honours', case=False, na=False).astype(int)

# --- Step 4: Create the target column 'field_of_study' using a comprehensive mapping ---
# Expanded mapping dictionary: more specific phrases first (sorted by length).
# Balanced keyword_to_field mapping:
keyword_to_field = {
    # IT and Computing
    'computer science': 'Computer Science',
    'computing': 'IT',
    'information systems': 'IT',
    'information technology': 'IT',
    'information security': 'Cybersecurity',
    'electronic commerce': 'IT',

    # Engineering disciplines (clear and general enough)
    'aerospace': 'Aerospace Engineering',
    'bioengineering': 'Bioengineering',
    'biomedical engineering': 'Bioengineering',
    'chemical': 'Chemical Engineering',
    'civil': 'Civil Engineering',
    'computer engineering': 'Computer Engineering',
    'electrical': 'Electrical Engineering',
    'environmental engineering': 'Environmental Engineering',
    'industrial and systems engineering': 'Industrial Engineering',
    'industrial engineering': 'Industrial Engineering',
    'systems engineering': 'Systems Engineering',
    'materials': 'Materials Engineering',
    'mechanical': 'Mechanical Engineering',
    'marine': 'Marine Engineering',
    'naval architecture': 'Marine Engineering',
    'mechatronics': 'Mechanical Engineering',
    'engineering science': 'Engineering Science',

    # Sciences (general yet accurate)
    'biomedical sciences': 'Biological Sciences',
    'biological sciences': 'Biological Sciences',
    'chemistry': 'Chemistry',
    'physics': 'Physics',
    'mathematics': 'Mathematics',
    'mathematics & economics': 'Mathematics & Economics',
    'computational biology': 'Computational Biology',
    'environmental earth systems': 'Environmental Science',

    # Business-related degrees
    'accountancy': 'Accountancy',
    'business analytics': 'Business Analytics',
    'business': 'Business',
    'finance': 'Finance',
    'marketing': 'Marketing',
    'supply chain management': 'Supply Chain Management',
    'hospitality': 'Hospitality Management',
    'culinary arts': 'Culinary Arts',

    # Healthcare-related
    'medicine': 'Medicine',
    'surgery': 'Medicine',
    'nursing': 'Nursing',
    'pharmacy': 'Pharmacy',
    'occupational therapy': 'Health Sciences',
    'physiotherapy': 'Health Sciences',
    'radiography': 'Health Sciences',
    'radiation therapy': 'Health Sciences',
    'dentistry': 'Dentistry',
    'nutrition': 'Food Science and Nutrition',
    'traditional chinese medicine': 'Traditional Chinese Medicine',

    # Social sciences and Humanities
    'communication': 'Communication Studies',
    'criminology': 'Criminology',
    'psychology': 'Psychology',
    'public policy': 'Public Policy',
    'global affairs': 'Global Affairs',
    'sociology': 'Sociology',
    'economics': 'Economics',
    'social sciences': 'Social Sciences',
    'linguistics': 'Linguistics',
    'education': 'Education',
    'early childhood': 'Education',
    'human resource': 'Human Resource Management',
    'social work': 'Social Work',

    # Law
    'law': 'Law',

    # Arts and Design
    'architecture': 'Architecture',
    'industrial design': 'Industrial Design',
    'fine arts': 'Fine Arts',
    'digital arts': 'Digital Arts',
    'game design': 'Game Design',
    'music': 'Music',
    'art, design': 'Art & Design',
    'interior design': 'Interior Design',

    # Built environment & Real estate
    'real estate': 'Real Estate',
    'facilities management': 'Facilities Management',

    # General fields and fallback
    'sports science': 'Sports Science',
    'maritime': 'Maritime Studies',
    'history': 'History',
    'english': 'English',
    'philosophy': 'Philosophy',
    'chinese': 'Chinese Studies',
    'applied science': 'Applied Science',
    'environmental studies': 'Environmental Studies',
}
keyword_to_field = {
    # Business and Management
    'accountancy': 'Business and Management',
    'business': 'Business and Management',
    'economics': 'Business and Management',
    'finance': 'Business and Management',
    'marketing': 'Business and Management',
    'management': 'Business and Management',
    'hospitality': 'Business and Management',
    'human resource': 'Business and Management',
    'supply chain': 'Business and Management',

    # Computer Science and IT
    'computer': 'Computer Science and IT',
    'computing': 'Computer Science and IT',
    'information systems': 'Computer Science and IT',
    'data science': 'Computer Science and IT',
    'artificial intelligence': 'Computer Science and IT',
    'software': 'Computer Science and IT',
    'cybersecurity': 'Computer Science and IT',
    'electronic commerce': 'Computer Science and IT',

    # Engineering
    'engineering': 'Engineering',
    'aerospace': 'Engineering',
    'bioengineering': 'Engineering',
    'chemical': 'Engineering',
    'civil': 'Engineering',
    'electrical': 'Engineering',
    'mechanical': 'Engineering',
    'environmental': 'Engineering',
    'materials': 'Engineering',
    'maritime': 'Engineering',
    'offshore': 'Engineering',
    'naval': 'Engineering',
    'systems': 'Engineering',
    'telematics': 'Engineering',

    # Arts and Humanities
    'arts': 'Arts and Humanities',
    'chinese': 'Arts and Humanities',
    'english': 'Arts and Humanities',
    'history': 'Arts and Humanities',
    'linguistics': 'Arts and Humanities',
    'philosophy': 'Arts and Humanities',
    'sociology': 'Arts and Humanities',
    'communication': 'Arts and Humanities',
    'criminology': 'Arts and Humanities',
    'public policy': 'Arts and Humanities',

    # Science
    'science': 'Science',
    'biological': 'Science',
    'biomedical': 'Science',
    'chemistry': 'Science',
    'physics': 'Science',
    'mathematical': 'Science',
    'environmental': 'Science',
    'sports': 'Science',
    'nutrition': 'Science',
    'food': 'Science',

    # Medicine and Health Sciences
    'medicine': 'Medicine and Health Sciences',
    'nursing': 'Medicine and Health Sciences',
    'pharmacy': 'Medicine and Health Sciences',
    'radiography': 'Medicine and Health Sciences',
    'physiotherapy': 'Medicine and Health Sciences',
    'occupational therapy': 'Medicine and Health Sciences',
    'radiation therapy': 'Medicine and Health Sciences',
    'dental': 'Medicine and Health Sciences',
    'biomedical sciences': 'Medicine and Health Sciences',

    # Law
    'law': 'Law',

    # Education
    'education': 'Education',

    # Design and Media
    'design': 'Design and Media',
    'media': 'Design and Media',
    'animation': 'Design and Media',
    'game design': 'Design and Media',
    'interior design': 'Design and Media',
    'communication design': 'Design and Media',

    # Social Sciences
    'psychology': 'Social Sciences',
    'sociology': 'Social Sciences',
    'public policy': 'Social Sciences',
    'global affairs': 'Social Sciences',

    # Mathematics and Statistics
    'mathematics': 'Mathematics and Statistics',
    'statistics': 'Mathematics and Statistics',
    'economics': 'Mathematics and Statistics',  # If combined with math

    # Others
    'music': 'Others',
    'fine arts': 'Others',
    'culinary': 'Others',
    'architecture': 'Others',
}

# Mapping function using keyword hierarchy
def map_to_field(degree):
    degree_lower = degree.lower()
    for keyword in sorted(keyword_to_field, key=len, reverse=True):
        if keyword in degree_lower:
            return keyword_to_field[keyword]
    return 'Others'

# Apply to DataFrame
df['field_of_study'] = df['degree_cleaned'].apply(map_to_field)



def map_to_field(degree):
    # Lowercase the degree text for matching
    degree = degree.lower()
    # Sort the keywords by length in descending order so that more specific phrases are matched first.
    for keyword in sorted(keyword_to_field.keys(), key=len, reverse=True):
        if keyword in degree:
            return keyword_to_field[keyword]
    return 'Others'

df['field_of_study'] = df['degree_cleaned'].apply(map_to_field)

# --- Step 5: (Optional) Check the updated unique standardized degree names ---
print("Unique standardized field of study labels:")
print(df['field_of_study'].unique())

# --- Step 6: Export and display the results ---
df.to_csv('poggers.csv', index=False)
print(df[['university', 'degree', 'degree_cleaned', 'is_honours', 'field_of_study']].head(10))



Unique standardized field of study labels:
['Business and Management' 'Engineering' 'Computer Science and IT'
 'Science' 'Design and Media' 'Arts and Humanities'
 'Mathematics and Statistics' 'Social Sciences'
 'Medicine and Health Sciences' 'Education' 'Others' 'Law']
  university                          degree           degree_cleaned  \
0        NTU  Bachelor of Accountancy (Hons)  bachelor of accountancy   
1        NTU  Bachelor of Accountancy (Hons)  bachelor of accountancy   
2        NTU     Bachelor of Business (Hons)     bachelor of business   
3        NTU     Bachelor of Business (Hons)     bachelor of business   
4        NTU           Aerospace Engineering    aerospace engineering   
5        NTU                  Bioengineering           bioengineering   
6        NTU                        Chemical                 chemical   
7        NTU            Computer Engineering     computer engineering   
8        NTU               Civil Engineering        civil engineering   
